In [1]:
from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner

agent = Agent(
  name="Joker",
  instructions="You are a helpful assistant.",
  model="o4-mini",
)

result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
async for event in result.stream_events():
  if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
    print(event.data.delta, end="", flush=True)

1. Why don’t skeletons fight each other?  
   They don’t have the guts!

2. What do you call fake spaghetti?  
   An impasta.

3. Why do bees have sticky hair?  
   Because they use honeycombs!

4. I told my computer I needed a break, and it said:  
   “Error 404: Coffee not found.”

5. I was going to tell you a time-travel joke,  
   but you didn’t like it.

In [ ]:
import random
from agents import Agent, Runner, ItemHelpers, function_tool

@function_tool
def how_many_jokes() -> int:
    """Returns a random number of jokes."""
    return random.randint(1, 10)

agent = Agent(
  name="Joker",
  instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
  tools=[how_many_jokes],
  model="o4-mini",
)

result = Runner.run_streamed(agent, input="Please tell me some jokes.")
print("=== Run starting ===")

async for event in result.stream_events():
  # We'll ignore the raw response events deltas
  if event.type == "raw_response_event":
    continue
  # When the agent updates, print that
  elif event.type == "agent_update_stream_event":
    print(f"Agent update: {event.new_agent.name}")
    continue
  # When items are generated, print them
  elif event.type == "run_item_stream_event":
    if event.item.type == "tool_call_item":
      print("-- Tool was called")
    elif event.item.type == "tool_call_output_item":
      print(f"-- Tool output: {event.item.output}")
    elif event.item.type == "message_output_item":
      print(f"-- Message output:\n{ItemHelpers.text_message_output(event.item)}")
    else:
      pass # Ignore other item types

print("=== Run complete ===")

=== Run starting ===
-- Tool was called
-- Tool output: 5
-- Message output:
Here are five jokes for you:

1. Why did the scarecrow win an award?  
   Because he was outstanding in his field!

2. What do you call fake spaghetti?  
   An impasta!

3. Why don’t skeletons fight each other?  
   They don’t have the guts.

4. How does a penguin build its house?  
   Igloos it together.

5. Why did the math book look sad?  
   Because it had too many problems.
=== Run complete ===
